# 実践 Python 3 輪読会

PyLadies Tokyo 輪読会 #1 (2016-01-20)  
本家ページ：http://www.qtrac.eu/pipbook.html

# 生成に関するデザインパターン

- 一般的にオブジェクトを生成する際にはそのオブジェクトのコンストラクタを呼ぶ

※※　クラスとコンストラクタの説明が必要？　※※

In [20]:
class Dog:
    def __init__(self, name, owner=None):
        self.name = name
        self.owner = owner
    
    def run(self):
        return '走ってるよ'

        
dog1 = Dog(name='poti', owner='pyladies')
print(dog1.name)
print(dog1.run())

print()

dog2 = Dog(name='kuro', owner='pyladies')
print(dog2.name)
print(dog2.run())

dog2.name = 'shiro'
print(dog2.name)

poti
走ってるよ

kuro
走ってるよ
shiro


In [ ]:
import pandas as pd

df = pd.DataFrame()

df.merge
df.sort

In [16]:
# オブジェクトを生成する一例
#（正確にはPythonの__new__や__init__はコンストラクタではない）

class Diagram():
    pass

# Personオブジェクトを生成
diagram = Diagram()
diagram

In [2]:
diagram = Diagram
diagram

__main__.Diagram

- オブジェクトの生成方法をもっと柔軟にしたい → 生成に関するデザインパターンが便利 (本章で紹介)
- 中にはPythonに関係ないデザインパターン (=C++向け) もあるのでそちらも確認

## 1.1 Abstract Factory パターン + α

- 他のオブジェクトから構成される複雑なオブジェクトを構成する際に利用するパターン  
- 整合性を必要とされるオブジェクト群を間違いなく生成することが出来る

なんのこっちゃと思うので例を見ていきます

### 1.1.1 古典的な Abstract Factory

図形を生成するプログラムについて考える (diagram1.py)  
ここでは以下の2つの Factory を考える

- プレーンテキストの図形を生成するDiagramFactory
- SVG形式の図形を生成するSVGDiagramFactory

In [3]:
def main():
    txt_diagram = create_diagram(TxtDiagramFactory())  # プレーンテキストの図形を生成
    txt_diagram.save("img/diagram.txt")                             # 生成した図形を保存
    print("wrote img/diagram.txt")

    svg_diagram = create_diagram(SvgDiagramFactory())  # SVG形式の図形を生成
    svg_diagram.save("img/diagram.svg")                             # 生成した図形を保存
    print("wrote img/diagram.svg")

両方で呼び出されている `create_diagram` って何？

In [4]:
def create_diagram(factory):                                                 # 1
    diagram = factory.make_diagram(30, 7)                          # 2
    text = factory.make_text(7, 3, "Abstract Factory")      # 3
    diagram.add(text)                                                                # 4
    return diagram                                                                      # 5

In [ ]:
def create_diagram():
    diagram = TxtDiagram(30, 7)
    text = TxtText(7, 3, "Abstract Factory")
    diagram.add(text)
    return diagram

どんな流れになっているのかというと，

1. 何かしらの図形ファクトリーを受け取って
2. 図形ファクトリーの `make_diagram` メソッドを使って指定したサイズで図形を作成
3. 指定したサイズで「Abstract Factory」という文字を生成
4. 生成した文字を図形に追加
5. 図形が完成．生成した図形を呼び出し元に返す

肝心の図形ファクトリーはどうなっている？

In [ ]:
class TxtDiagramFactory:
    def make_diagram(self, width, height):
        return TxtDiagram(width, height)

    def make_text(self, x, y, text, fontsize=12):
        return TxtText(x, y, text, fontsize)

    
class SvgDiagramFactory(DiagramFactory):
    def make_diagram(self, width, height):
        return SvgDiagram(width, height)

    def make_text(self, x, y, text, fontsize=12):
        return SvgText(x, y, text, fontsize)

図形ファクトリーはどちらも同じインタフェースを提供している  
→ 同じメソッドを提供している ( `make_diagrame` ，  `make_text` )

しかしどちらも呼び出された後に返しているオブジェクトが違う  
→ `TxtDiagramFactory` だと `TxtDiagram` ，`SvgDiagramFactory` だと `SvgDiagram`

__何が嬉しいのか？__

- どちらの図形ファクトリーも同じインタフェースを利用しているので，呼び出す側は両者の違いを意識しなくても良い
  - `create_diagram` の中身を改めて確認
- `TxtDiagramFactory` を利用しているときには `TxtDiagram` ， `SvgDiagramFactory` を利用しているときには `SvgDiagram` など，意識しなくても対応したオブジェクトを利用できる

In [ ]:
# TxtDiagramやSvgDiagramの実装は全く違う！
# （一部省略）

txt_diagram = TxtDiagram()
txt_diagram.width

class TxtDiagram:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.diagram = _create_rectangle(self.width, self.height, BLANK)

    def add(self, component):
        for y, row in enumerate(component.rows):
            for x, char in enumerate(row):
                self.diagram[y + component.y][x + component.x] = char
                
class SvgDiagram:
    def __init__(self, width, height):
        pxwidth = width * SVG_SCALE
        pxheight = height * SVG_SCALE
        self.diagram = [SVG_START.format(**locals())]
        outline = SvgRectangle(0, 0, width, height, "lightgreen", "black")
        self.diagram.append(outline.svg)

    def add(self, component):
        self.diagram.append(component.svg)

※※　ここで需要があれば `Abstract Factory` パターンを使わなかった場合のデモする　※※

### 1.1.2 パイソニックな Abstract Factory

上記のコードは良い例だが，いくつか欠点がある

1. ファクトリーのインスタンスは実際に生成する必要がない
2. `TxtDiagramFactory` と `SvgDiagramFactory` のコードがほとんど同じ
3. 最上位の名前空間にすべてのクラスが存在する (e.g. TxtDiagram, TxtRectangle)

どうすれば良いのか？ (`diagram2.py` を確認しつつ)

1. `make_diagram` などのメソッドをクラスメソッドにしてしまう（`@classmethod` を利用する．第一引数は `Class` になる）
2. `SvgDiagramFactory` で `TxtDiagramFactory` を継承する + 3の変更
3. `TxtDiagram`，`TxtRectangle` といったクラスを `TxtDiagramFactory` の中に入れてしまう (更に `Txt` という `prefix` をとる．`SvgDiagramFactory` についても同様)

--------

__インスタンスメソッド / クラスメソッド__

In [25]:
class Dog:
    animal_type = 'mammalia'
    
    # self.*** はインスタンスメソッドへのアクセス
    def __init__(self, name, owner):
        self.name = name
        self.owner = owner
        
    @classmethod
    def bark(cls):
        return 'bow-wow'
    
    def get_owner(self):
        return self.owner
    
    def change_owner(self, owner):
        self.owner = owner

In [26]:
dog1 = Dog('pochi', 'tokyo')
dog1.change_owner('kyoto')
dog1.owner

'kyoto'

In [4]:
dog = Dog  # インスタンスを生成していない

print('animal_type: {0}'.format(dog.animal_type))
print('bark: {0}'.format(dog.bark()))
print('name: {0}'.format(dog.name))

animal_type: mammalia
bark: bow-wow


AttributeError: type object 'Dog' has no attribute 'name'

In [5]:
print('owner: {0}'.format(dog.get_owner()))

TypeError: get_owner() missing 1 required positional argument: 'self'

In [6]:
dog = Dog(name='pochi', owner='Ai')  # インスタンスを生成している

print('animal_type: {0}'.format(dog.animal_type))
print('bark: {0}'.format(dog.bark()))
print('name: {0}'.format(dog.name))
print('owner: {0}'.format(dog.get_owner()))

animal_type: mammalia
bark: bow-wow
name: pochi
owner: Ai


In [7]:
import sys

print(sys.getsizeof(Dog))
print(sys.getsizeof(dog))

976
56
